In [ ]:
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors

### Lectura del fichero CSV

Vamos a cargar el fichero, que previamente hemos ingestado en HDFS, en un RDD de Spark, con el formato conveniente para nuestros objetivos.

In [ ]:
lines = sc.textFile('/user/cloudera/T_F_DR14_ZooSpec_10000.csv')

In [ ]:
# Vemos que hay 10.001 filas en el RDD. Esto quiere decir que incluye el header o cabecera
lines.count()

In [ ]:
# Vamos a desechar el header
lines_f = lines.zipWithIndex().filter(lambda tup: tup[1] > 0).map(lambda x: x[0])
lines_f.count()

In [ ]:
# Convertimos las lineas de texto separado por comas en un DataFrame
rows = lines_f.map(lambda l: l.split(","))

def build_features_bis(p):
    return (p[0], int(p[1]), Vectors.dense([float(e) for e in p[2:]]),)

data = rows.map(build_features_bis)
df = sqlContext.createDataFrame(data, ['dr7objid', 'target', 'features'])

In [ ]:
df.show(1)

In [ ]:
# Vemos el número de filas y de columnas que tiene el DataFrame
print('Número de filas (imágenes): {}'.format(df.count()))
print('Número de columnas (id + target + features): {}'.format(len(df.columns)))

### Filtrado de imágenes no clasificadas

Para el entrenamiento no nos hacen falta todos los datos, nos basta con los datos de aquellas imágenes que se han clasificado satisfactoriamente. Así pues, vamos a descartar aquellas imágenes cuyo campo `target` tiene valor `0`.

In [ ]:
# Vemos cuántas imágenes hay de cada tipo
# 0 = incierto
# 1 = elíptica
# 2 = espiral
df.groupBy('target').count().show()

In [ ]:
# Construímos un nuevo DataFrame solamente con las imágenes clasificadas
labeled_df = df.filter(df['target'] != 0)

In [ ]:
# Vemos que el número de imágenes seleccionadas es coherente con la query anterior
labeled_df.count()

### Reducción de los datos: Principal Component Analysis (PCA)

Los datos de los atributos, no es el ideal para entrenar un algoritmo de clasificación:
* **es muy grande** 3701 filas * 4096 columnas ~ 15M de celdas
* **es poco denso** hay pocas muestras (3701 imágenes) para el número de atributos (4096 píxeles). De intentar aplicar algunos algoritmos de clasificación sobre este conjunto de datos, podríamos incurrir en la [maldición de la dimensión (en inglés)](https://en.wikipedia.org/wiki/Curse_of_dimensionality)

Para solucionar ambos problemas utilizaremos el método PCA para reducir el número de atributos.

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
pca = PCA(k=64, inputCol='features', outputCol='pca_features')
model = pca.fit(labeled_df)
pca_features = model.transform(labeled_df).collect()